In [17]:
import cv2
import skimage
import numpy as np
import time

In [3]:
img = cv2.imread('test_image/soccer.jpg')

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

grad_x = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize = 3)
grad_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize = 3)

sobel_x = cv2.convertScaleAbs(grad_x)
sobel_y = cv2.convertScaleAbs(grad_y)

edge_strength = cv2.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0) # addWeighted ==> sobel_x, sobel_y 가중치 값 합치기.

cv2.imshow('Original', gray)
cv2.imshow('sobel_x', sobel_x)
cv2.imshow('sobel_y', sobel_y)
cv2.imshow('edge strength', edge_strength)

cv2.waitKey()
cv2.destroyAllWindows()

In [4]:
canny1 = cv2.Canny(gray, 50, 150)
canny2 = cv2.Canny(gray, 100, 200)

cv2.imshow('original', gray)
cv2.imshow('canny1', canny1)
cv2.imshow('canny2', canny2)

cv2.waitKey()
cv2.destroyAllWindows()

In [7]:
canny = cv2.Canny(gray, 100, 200)

contour, hierarchy = cv2.findContours(canny, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE) # CHAIN_APPROX_NONE 이 모든 점 기록.

lcontour = []

for i in range(len(contour)):
    if contour[i].shape[0] > 100: # 길이 50 이상인 것만. # 갔다가 오는 거 기록해서 100으로.
        lcontour.append(contour[i])


cv2.drawContours(img, lcontour, -1, (0,255,0), 3)


cv2.imshow('original with contours', img)
cv2.imshow('canny', canny)

cv2.waitKey()
cv2.destroyAllWindows()

In [13]:
### 허프 변환 활용 명암 원 검출

img = cv2.imread('test_image/apples.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

apples = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 200, param1 = 150, param2 = 20, minRadius = 50, maxRadius = 120)
## 검출된 원들 사이 최소거리(원 사이 최소거리)=200, canny edge 임계값 = 150, 투표 임계값 = 20, 원 최소 반지름 50, 최대 반지름 120


for i in apples[0]:
    cv2.circle(img, (int(i[0]), int(i[1])), int(i[2]), (255, 0, 0), 2)

cv2.imshow('Apple detection', img)

cv2.waitKey()
cv2.destroyAllWindows()

In [16]:
img = skimage.data.coffee()
cv2.imshow('coffee image', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

slic1 = skimage.segmentation.slic(img, compactness = 20, n_segments = 600) # compactness ==> 모양새 조절. 높음 높을수록 다각형 모양.
### 높으면 높을 수록 색상 균일성 딱 안 좋아짐.
sp_img1 = skimage.segmentation.mark_boundaries(img, slic1)
sp_img1 = np.uint8(sp_img1 * 255.0)

slic2 = skimage.segmentation.slic(img, compactness = 40, n_segments = 600)
sp_img2 = skimage.segmentation.mark_boundaries(img, slic2)
sp_img2 = np.uint8(sp_img2 * 255.0)

cv2.imshow('super pixels (compact-20)', cv2.cvtColor(sp_img1, cv2.COLOR_RGB2BGR))
cv2.imshow('Super pixels (compact-40)', cv2.cvtColor(sp_img2, cv2.COLOR_RGB2BGR))

cv2.waitKey()
cv2.destroyAllWindows()

In [24]:
coffee = skimage.data.coffee()

start = time.time()
slic = skimage.segmentation.slic(coffee, compactness = 20, n_segments = 600, start_label = 1)

g = skimage.graph.rag_mean_color(coffee, slic, mode = 'similarity')
ncut = skimage.graph.cut_normalized(slic, g) # ncut 사용 최적화.
print(coffee.shape, '분할하는데 걸리는 시간 ', time.time() - start)

marking = skimage.segmentation.mark_boundaries(coffee, ncut)
ncut_coffee = np.uint8(marking * 255.0)

cv2.imshow('normalized cut', cv2.cvtColor(ncut_coffee, cv2.COLOR_RGB2BGR))

cv2.waitKey()
cv2.destroyAllWindows()

### 영역 분할 결과물 보면 스푼 쪽 좀 많이 요상함. 색상 단위로만 봐서 쫌 맛이 가 있음.
# 그래서 딥러닝으로 갔다.

(400, 600, 3) 분할하는데 걸리는 시간  6.349429368972778
